**Authors:**\
Samuel Reyes Sanz\
Eduardo Miguel Riederer

# Multi-Author Writing Style Analysis 2023

The goal of the style change detection task is to identify text positions within a given multi-author document at which the author switches.

The simultaneous change of authorship and topic will be carefully controlled and we will provide participants with datasets of three difficulty levels:

- Easy: The paragraphs of a document cover a variety of topics, allowing approaches to make use of topic information to detect authorship changes.
- Medium: The topical variety in a document is small (though still present) forcing the approaches to focus more on style to effectively solve the detection task.
- Hard: All paragraphs in a document are on the same topic.

All documents are provided in English and may contain an arbitrary number of style changes. However, style changes may only occur between paragraphs (i.e., a single paragraph is always authored by a single author and contains no style changes).

In [1]:
import os
import json
import torch
import random
import numpy as np
import transformers
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score
from transformers import AutoModel, AutoTokenizer
from transformers.models.deberta_v2 import DebertaV2TokenizerFast
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

/home/samu/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Constants & Parameters

In [2]:
# Edu
#PATH = 'C:\\Users\\nobody\\Downloads\\Master\\NLP\\pan23-multi-author-analysis'
#TRAIN_PATH = os.path.join(PATH, 'data\\pan23-multi-author-analysis-dataset1\\pan23-multi-author-analysis-dataset1-train')
#TEST_PATH = os.path.join(PATH, 'data\\pan23-multi-author-analysis-dataset1\\pan23-multi-author-analysis-dataset1-validation')

# Samu
PATH = '/home/samu/workspace/projects/UPM/NLP/pan23-multi-author-analysis'
TRAIN_PATH = os.path.join(PATH, 'data/pan23-multi-author-analysis-dataset3/pan23-multi-author-analysis-dataset3-train')
TEST_PATH = os.path.join(PATH, 'data/pan23-multi-author-analysis-dataset3/pan23-multi-author-analysis-dataset3-validation')

TRAIN_BATCH_SIZE = 24
VAL_BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 2e-5
MAX_LEN = 512
TOKENIZER = DebertaV2TokenizerFast.from_pretrained("microsoft/deberta-v3-base")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/samu/.conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### Using GPU

In [3]:
# Setting up the device for GPU usage

from torch import cuda

device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

## Data processing

Se separan los párrafos con el token [SEP] y se colocan por pares en un DataFrame junto a su etiqueta: 
- 0: no ha cambiado el autor
- 1: ha cambiado el autor


Si un texto tiene 4 párrafos (ABCD) se dividirá en 3 instancias: AB, BC, CD

In [4]:
def load_data(folder_path):
    text_files = [f for f in os.listdir(folder_path) if f.startswith('problem') and f.endswith('.txt')]
    rows = []

    for text_file in text_files:
        text_file_path = os.path.join(folder_path, text_file)
        label_file_path = text_file_path.replace('.txt', '.json').replace('problem', 'truth-problem')

        with open(text_file_path, 'r', encoding="utf8") as file:
            paragraphs = file.read().split('\n')

        with open(label_file_path, 'r', encoding="utf8") as file:
            labels = json.load(file)['changes']

        for i in range(len(labels)):
            combined_text = paragraphs[i] + ' [SEP] ' + paragraphs[i+1] 
            label = labels[i] 
            rows.append({'Text': combined_text, 'Label': label})

    return pd.DataFrame(rows)

In [5]:
# Loading training and test data
train_df = load_data(TRAIN_PATH)
test_df = load_data(TEST_PATH)

In [6]:
train_df.Label.value_counts()

Label
0    10092
1     9021
Name: count, dtype: int64

### Limpieza del conjunto de datos

Se detecta que existen carácteres especiales como emojis o letras chinas. Deberta usa sentencepiece tokenizer que soporta diferentes idiomas y carácteres especiales como emojis (https://github.com/google/sentencepiece).

In [7]:
def find_special_characters(df):
    special_chars = set()
    texts_withs_special_characters = []
    
    for index, text in enumerate(df['Text']):
        for char in text:
            if not char.isascii():
                special_chars.add(char)
                texts_withs_special_characters.append(index)
                
                if char == "🥴":
                    print (index)
                    
    return (list(special_chars), texts_withs_special_characters)

special_chars, texts_withs_special_characters = find_special_characters(train_df)

16612
16613


In [8]:
print(special_chars)

['⚡', '遼', '\u202f', '茶', '…', 'ú', 'П', 'ı', '’', 'ż', 'ṣ', '国', 'ç', '️', '天', '́', '夏', '—', '·', 'ã', 'κ', 'ḗ', '愈', 'í', '🌍', 'ł', '🤍', '😂', 'â', 'è', 'ʿ', '🗳', '🙂', '客', '取', 'К', 'н', '劉', 'ȝ', '©', 'ü', '規', '≠', 'э', '馬', 'Χ', '₂', 'з', 'Н', '€', '🤯', 'с', '\u2003', '\u200b', 'ι', 'ġ', '🏡', 'Γ', '宗', 'τ', '成', 'п', 'á', '¢', 'ē', 'Ç', '⚖', '銃', '覺', '“', '\xad', '愛', '⅓', 'ś', '\u2002', 'ş', '篇', '店', '¹', '«', '£', '§', '棣', 'ū', 'ō', '•', '🤙', 'ṇ', 'и', 'ä', 'Ε', 'ī', '號', 'Ō', '登', 'ʰ', 'ó', '貞', '邦', 'đ', 'ð', '性', '🥴', '秋', '🙃', 'к', '國', '清', '慧', 'ο', '翱', 'Ü', '年', 'ῑ', '中', 'ğ', '封', '🏼', '‘', '₁', 'т', '可', 'ц', '😑', '🤣', '祖', '卑', '„', 'ˈ', '周', 'е', '名', '永', '×', 'ö', '喫', 'σ', '總', '基', 'č', '唐', '🤬', '–', 'л', '西', '❗', '”', '民', '민', '🙏', 'ы', 'ô', '‡', '🗽', '司', '⅔', 'Ř', '😷', 'а', 'î', 'ك', '¿', '😔', '♂', '涨', 'ч', 'ʒ', '\U0001fae2', '高', '🏻', '韓', 'ñ', '\u2060', 'ή', '🇸', '義', 'Ṛ', 'þ', '🇨', '⬇', 'ý', '🤔', 'å', 'о', '족', 'ɔ', 'ø', '樂', '資', '朱', '̂', '☠', 'Š

In [9]:
#Numero de textos con caracteres especiales

texts_withs_special_characters = sorted(set(texts_withs_special_characters))
texts_withs_special_characters
len(texts_withs_special_characters)

5612

Podemos ver que tokeniza el texto del emoji.

In [10]:
tokenization = TOKENIZER.encode_plus(
            text = "😐",
            add_special_tokens=True,
            max_length=MAX_LEN,
            pad_to_max_length=True,
            return_token_type_ids=True
            )

tokenization

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/samu/.conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{'input_ids': [1, 507, 125322, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### Función de truncación

Dado que el tamaño máximo de entrada del modelo DeBERTa-v3 es 512 tokens, comprobamos el número de textos con más de 500 palabras. Para los pocos casos en los que se supere el tamño de entrada, utilizaremos **Transition-Focused Truncation** para recortar las entradas. En las entradas que sean menores a 512 tokens, se aplicará padding. 

In [11]:
sum(len(text.split()) > 512 for text in train_df['Text'])

12

In [12]:
def transition_focused_truncation(text, tokenizer, max_tokens = MAX_LEN, sep_token="[SEP]"):
    
    tokens = tokenizer.tokenize(text)

    # CASE WHERE TOKENS DO NOT EXCEED LIMIT
    if len(tokens) <= max_tokens:
        return tokens


    # CASE WHERE TOKENS EXCEED LIMIT
   
    # Find the index of the [SEP] token
    sep_index = tokens.index(sep_token) if sep_token in tokens else -1
    
    # If [SEP] is not found, truncate the first max_tokens
    if sep_index == -1:
        return tokens[:max_tokens]

    # Let's check what text is longer
    text_1 = tokens[:sep_index]
    text_2 = tokens[(sep_index + 1):]
    
    if (len(text_1) >= max_tokens // 2):
        
        #Case where both texts are bigger than max_tokens / 2
        if (len(text_2) >= max_tokens // 2):
            text_1 = text_1[int(len(text_1)-(max_tokens//2)):]
            
            if int(len(text_2) - (max_tokens//2)) != 0:
                text_2 = text_2[:-int(len(text_2) - (max_tokens//2))]
            else:
                text_2 = text_2[:]
                
            if max_tokens % 2 == 0:
                rand_num = random.randint(0, 1)
                if rand_num == 0:
                    text_1 = text_1[1:]
                else:
                    text_2 = text_2[:-1]
                
        #Case where only text_1 is bigger than max_tokens / 2
        else:
            max_tokens_for_text = max_tokens - len(text_2)
            text_1 = text_1[-max_tokens_for_text:]
            
            if (len(text_1) + len(text_2) == max_tokens):
                text_1 = text_1[1:]
                
    #Case where only text_2 is bigger than max_tokens / 2
    else:
        max_tokens_for_text = max_tokens - len(text_1)
        text_2 = text_2[:max_tokens_for_text]
        
        if (len(text_1) + len(text_2) == max_tokens):
            text_2 = text_2[:-1]
        
    return (text_1 + [sep_token] + text_2)

In [13]:
# Example usage

long_text = train_df["Text"][781]
tokens = transition_focused_truncation(long_text, TOKENIZER)
"".join(tokens).replace("▁", " ")[1:-1]

'When Russia says Nazi they are not engaging in doublespeak, where they accuse the enemy of doing exactly what they do, themselves. To them "Nazi" simply means "western enemy of russia." Russians don\'t care about torture or concentration camps, why would they? It\'s their preferred solution for political differences. Even russian mercenaries ride into battle with nazi symbology tattooed on them.[SEP] I don\'t know if I was clear. I am saying people perceive russia is engaging in nazi tactics while claiming they hate nazis, thus engaging in doublespeak. But this is not how russia sees it because the nazis tactics were not the defining quality of what made them nazi'

## Dataset
Se crea el dataset personalizado en el que se devuelve una instancia de texto tokenizado, junto a la máscara, los token type ids y las etiquetas.

In [14]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len=MAX_LEN):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.Text
        self.targets = self.data.Label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):

        text = str(self.text[index])
        
        truncated_text = transition_focused_truncation(text, self.tokenizer, self.max_len, "[SEP]")

        batch_encoder = self.tokenizer(
            truncated_text,
            is_split_into_words=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=True,
            return_tensors='pt'
        )

        ids = batch_encoder['input_ids']
        mask = batch_encoder['attention_mask']
        token_type_ids = batch_encoder["token_type_ids"]

        return {
            'ids': ids.squeeze(0),
            'mask': mask.squeeze(0),
            'token_type_ids': token_type_ids.squeeze(0),
            'targets':self.targets[index]
        }

In [15]:
training_dataset = CustomDataset(dataframe=train_df, tokenizer=TOKENIZER, max_len=MAX_LEN)
testing_dataset = CustomDataset(dataframe=test_df, tokenizer=TOKENIZER, max_len=MAX_LEN)

training_loader = DataLoader(training_dataset, batch_size=VAL_BATCH_SIZE, shuffle=True)
testing_loader = DataLoader(testing_dataset, batch_size=VAL_BATCH_SIZE, shuffle=True)

print("TRAIN Dataset: {}".format(len(training_dataset)))
print("TEST Dataset: {}".format(len(testing_dataset)))

TRAIN Dataset: 19113
TEST Dataset: 4112


## Creating the model to fine-tune

Se añade la cabeza de clasificación. Se ha utilizado el modelo DebertaV3 (https://arxiv.org/abs/2111.09543), una versión mejorada del modelo Deberta gracias a entrenar el modelo utilizando replaced token detection (RTD), una técnica más sofisticada que emplear masked language modeling (MLM). Este modelo admite como entrada un tamaño de secuencia de 512 y devuelve un tamaño de embedding de 768. Se probó a emplear la versión large de Deberta V3, pero se descartó su uso ya que proporciona resultados muy similares, pero aumenta en gran medida el tiempo de aprendizaje y los requisitos de memoria GPU.

Dada la salida del modelo se añaden las siguientes capas:
- Inicialmente se realiza la media de todos los embeddings de la secuencia, generando un embedding que representa a los dos párrafos.
- Utilizando este embedding, se procesa a través de una cabeza de clasificación compuesta por capas densas, de normalización, de dropout y funciones de activación GELU.

In [16]:
class MeanPooling(torch.nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

In [17]:
# Creating the customized model, by adding a drop out and a dense layer on top of distilBERT to get the final 
# output for the model.

class DeBERTaClass(torch.nn.Module):
    
    def __init__(self):
        super(DeBERTaClass, self).__init__()
        self.dropout = 0.2
        self.hidden_embd = 768
        self.output_layer = 1
        
        self.l1 = AutoModel.from_pretrained('microsoft/deberta-v3-base')

        self.pool = MeanPooling()
        self.head = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_embd, 128),
            torch.nn.BatchNorm1d(128),
            torch.nn.GELU(),
            torch.nn.Dropout(self.dropout),
            torch.nn.Linear(128, 32),
            torch.nn.BatchNorm1d(32),
            torch.nn.GELU(),
            torch.nn.Dropout(self.dropout),
            torch.nn.Linear(32, self.output_layer)
        )

    def forward(self, ids, mask, token_type_ids):
        output_1 = self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False) #bs, sl, es
        last_hidden_states = output_1[0]
        feature = self.pool(last_hidden_states, mask)
        output = self.head(feature)

        return output.squeeze(-1)

model = DeBERTaClass()
model.to(device)

2023-11-29 18:22:42.998932: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-29 18:22:43.016662: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-29 18:22:43.016677: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-29 18:22:43.016693: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-29 18:22:43.021217: I tensorflow/core/platform/cpu_feature_g

DeBERTaClass(
  (l1): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
              (dropo

La función de pérdida utilizada será la entropía cruzada binaria, usando la versión "with logits" para evitar tener que aplicar una capa con la función sigmoide.

In [18]:
def loss_fn(outputs, targets):
    return torch.nn.functional.binary_cross_entropy_with_logits(outputs, targets)

El optimizador empleado es Adam, uno de los más populares.

In [19]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

## Entrenamiento

Con epochs fijas

In [20]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):

        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float) # bs,

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets) # bs, sl, 1 / bs

        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [21]:
"""
for epoch in range(EPOCHS):
    train(epoch)
"""

'\nfor epoch in range(EPOCHS):\n    train(epoch)\n'

Con early stopping

In [22]:
class EarlyStopping:
    
    def __init__(self, patience=3, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_metric = None
        self.epochs_no_improve = 0
        self.should_stop = False

    def __call__(self, val_metric):
        if self.best_metric == None:
            self.best_metric = val_metric
        elif self.best_metric - val_metric > self.min_delta:
            self.best_metric = val_metric
            self.epochs_no_improve = 0
        else:
            self.epochs_no_improve += 1
            if self.epochs_no_improve >= self.patience:
                self.should_stop = True

Validación del modelo.

In [23]:
def validate(validation_loader, model, loss_fn, device):
    
    # Poner el modelo en modo de evaluación
    model.eval()

    total_loss = 0
    all_predictions = []
    all_targets = []

    # Desactivar el cálculo del gradiente
    with torch.no_grad():
        for _, data in enumerate(validation_loader, 0):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.float)  # bs,

            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            total_loss += loss.item()

            # Suponiendo una tarea de clasificación, convertir las salidas a predicciones
            predictions = torch.round(torch.sigmoid(outputs)).squeeze()  # Convertir logits a probabilidades y luego a clases binarias
            all_predictions.extend(predictions.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    avg_loss = total_loss / len(validation_loader)
    accuracy = (np.array(all_predictions) == np.array(all_targets)).mean()
    f1 = f1_score(all_targets, all_predictions)
    
    return avg_loss, accuracy, f1


Comienza el entrenamiento. Se guarda el modelo con mayor F1 en validación.

In [24]:
best_f1_score = 0.0
best_model_state = None
early_stopping = EarlyStopping(patience=3, min_delta=0.01)

for epoch in range(EPOCHS):
    train(epoch)
    avg_loss, accuracy, f1 = validate(testing_loader, model, loss_fn, device)
    print(f'Epoch: {epoch},\nValidación - Avg Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}')

    if f1 > best_f1_score:
        best_f1_score = f1
        torch.save(model, 'modelo3-base-task3.pth')

    early_stopping(f1)

    if early_stopping.should_stop:
        print("Early stopping triggered")
        break


Epoch: 0, Loss:  0.7505934834480286
Epoch: 0,
Validación - Avg Loss: 0.6710, Accuracy: 0.5501, F1 Score: 0.6689
Epoch: 1, Loss:  0.7667965292930603
Epoch: 1,
Validación - Avg Loss: 0.4718, Accuracy: 0.7636, F1 Score: 0.7416
Epoch: 2, Loss:  0.5437997579574585
Epoch: 2,
Validación - Avg Loss: 0.4262, Accuracy: 0.7938, F1 Score: 0.7818
Epoch: 3, Loss:  0.32313966751098633
Epoch: 3,
Validación - Avg Loss: 0.4258, Accuracy: 0.7979, F1 Score: 0.7802
Epoch: 4, Loss:  0.194144606590271
Epoch: 4,
Validación - Avg Loss: 0.4929, Accuracy: 0.7977, F1 Score: 0.8055
Epoch: 5, Loss:  0.11288212239742279
Epoch: 5,
Validación - Avg Loss: 0.4771, Accuracy: 0.8067, F1 Score: 0.8111
Epoch: 6, Loss:  0.10796435922384262
Epoch: 6,
Validación - Avg Loss: 0.5213, Accuracy: 0.8011, F1 Score: 0.7993
Epoch: 7, Loss:  0.13444465398788452
Epoch: 7,
Validación - Avg Loss: 0.5804, Accuracy: 0.7921, F1 Score: 0.7767
Epoch: 8, Loss:  0.09689974039793015
Epoch: 8,
Validación - Avg Loss: 0.6358, Accuracy: 0.7967, F1 Sc

In [28]:
print("MEJOR F1 EN VALIDACIÓN", round(best_f1_score*100, 3), "%")

MEJOR F1 EN VALIDACIÓN 81.112 %
